In [1]:
!pip install mcp

  Using cached anyio-4.11.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached jsonschema-4.25.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached PyJWT-2.10.1-py3-none-any.whl.metadata (4.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.2 MB/s  0:00:00
Using cached anyio-4.11.0-py3-none-any.whl (109 kB)
Using cached jsonschema-4.25.1-py3-none-any.whl (90 kB)
Using cached PyJWT-2.10.1-py3-none-any.whl (22 kB)
  Attempting uninstall: uvicorn
    Found existing installation: uvicorn 0.30.6
    Uninstalling uvicorn-0.30.6:
      Successfully uninstalled uvicorn-0.30.6
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.8.0
    Uninstalling PyJWT-2.8.0:
      Successfully uninstalled PyJWT-2.8.0
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.23.4
    Uninstalling pydantic_core-2.23.4:
      Successfully uninstalled pydantic_core-2.23.4
  Attempting uninstall: anyio━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/8 [pydantic-core]
    

In [14]:
import json
import os
import snowflake.connector
from snowflake.connector import DictCursor
from dotenv import load_dotenv

In [15]:
load_dotenv()

SNOWFLAKE_CONFIG = {
    'user': os.getenv('SNOWFLAKE_USER'),
    'password': os.getenv('SNOWFLAKE_PASSWORD'),
    'account': os.getenv('SNOWFLAKE_ACCOUNT'),
    'warehouse': os.getenv('SNOWFLAKE_WAREHOUSE'),
    'database': os.getenv('SNOWFLAKE_DATABASE'),
    'schema': os.getenv('SNOWFLAKE_SCHEMA'),
    'role': os.getenv('SNOWFLAKE_ROLE', 'PUBLIC')
}

In [16]:
print("="*70)
print("LAB 8: MCP WITH DISCOVERY")
print("="*70)
print("\n🔑 Client discovers tools at RUNTIME, not design time!")

LAB 8: MCP WITH DISCOVERY

🔑 Client discovers tools at RUNTIME, not design time!


In [31]:
print("\n" + "="*70)
print("PART 1: CREATING MCP SERVER")
print("="*70)

class SnowflakeToolServer:
    """
    MCP Server that provides Snowflake data tools
    Server EXPOSES its capabilities via get_tool_descriptions()
    """
    
    def __init__(self, name: str, sf_config: dict):
        self.name = name
        self.config = sf_config
        self.tools = {}
        self._register_tools()
        print(f"\n🏭 MCP SERVER '{name}' initializing...")
        print(f"   Registering {len(self.tools)} tools internally")
    
    def _register_tools(self):
        """Register all available tools"""
        self.tools = {
            "query_database": self.query_database,
            "list_tables": self.list_tables,
            "get_table_schema": self.get_table_schema,
            "analyze_data": self.analyze_data
        }
    
    def query_database(self, sql: str) -> dict:
        """Execute SQL query and return results"""
        print(f"\n🔧 TOOL EXECUTING: query_database")
        print(f"   SQL: {sql[:100]}...")
        
        try:
            conn = snowflake.connector.connect(**self.config)
            cursor = conn.cursor(DictCursor)
            cursor.execute(sql)
            results = cursor.fetchall()
            cursor.close()
            conn.close()
            
            print(f"   ✅ Returned {len(results)} rows")
            return {
                "success": True,
                "rows": results,
                "row_count": len(results)
            }
        except Exception as e:
            print(f"   ❌ Error: {str(e)}")
            return {"success": False, "error": str(e)}
    
    def list_tables(self) -> dict:
        """List all tables in the database"""
        print(f"\n🔧 TOOL EXECUTING: list_tables")
        
        try:
            conn = snowflake.connector.connect(**self.config)
            cursor = conn.cursor()
            cursor.execute("SHOW TABLES")
            tables = [row[1] for row in cursor.fetchall()]
            cursor.close()
            conn.close()
            
            print(f"   ✅ Found {len(tables)} tables")
            return {"success": True, "tables": tables}
        except Exception as e:
            print(f"   ❌ Error: {str(e)}")
            return {"success": False, "error": str(e)}
    
    def get_table_schema(self, table_name: str) -> dict:
        """Get schema for a specific table"""
        print(f"\n🔧 TOOL EXECUTING: get_table_schema({table_name})")
        
        try:
            conn = snowflake.connector.connect(**self.config)
            cursor = conn.cursor()
            cursor.execute(f"DESCRIBE TABLE {table_name}")
            schema = [
                {"name": row[0], "type": row[1], "nullable": row[2]}
                for row in cursor.fetchall()
            ]
            cursor.close()
            conn.close()
            
            print(f"   ✅ Found {len(schema)} columns")
            return {"success": True, "schema": schema}
        except Exception as e:
            print(f"   ❌ Error: {str(e)}")
            return {"success": False, "error": str(e)}
    
    def analyze_data(self, table_name: str, question: str) -> dict:
        """Use Cortex to analyze data from a table"""
        print(f"\n🔧 TOOL EXECUTING: analyze_data")
        print(f"   Table: {table_name}")
        print(f"   Question: {question}")
        
        try:
            conn = snowflake.connector.connect(**self.config)
            cursor = conn.cursor(DictCursor)
            cursor.execute(f"SELECT * FROM {table_name} LIMIT 5")
            sample_data = cursor.fetchall()
            
            prompt = f"""
            Table: {table_name}
            Sample data: {json.dumps(sample_data[:3], default=str)}
            
            Question: {question}
            
            Provide a brief analysis.
            """
            
            cortex_query = f"""
            SELECT SNOWFLAKE.CORTEX.COMPLETE(
                'mistral-large2',
                '{prompt.replace("'", "''")}'
            ) as analysis
            """
            
            cursor.execute(cortex_query)
            result = cursor.fetchone()
            analysis = result['ANALYSIS'] if result else "No analysis available"
            
            cursor.close()
            conn.close()
            
            print(f"   ✅ Analysis complete")
            return {"success": True, "analysis": analysis}
        except Exception as e:
            print(f"   ❌ Error: {str(e)}")
            return {"success": False, "error": str(e)}
    
    def get_tool_descriptions(self) -> list:
        """
        🌟 KEY METHOD: Server exposes its capabilities
        This is what Client calls to DISCOVER tools!
        """
        print(f"\n📋 SERVER: Responding to discovery request from client...")
        print(f"   Exposing {len(self.tools)} tool descriptions")
        
        return [
            {
                "name": "query_database",
                "description": "Execute a SQL query on Snowflake database",
                "parameters": {"sql": "string - SQL query to execute"}
            },
            {
                "name": "list_tables",
                "description": "List all tables in the database",
                "parameters": {}
            },
            {
                "name": "get_table_schema",
                "description": "Get the schema/structure of a specific table",
                "parameters": {"table_name": "string - name of the table"}
            },
            {
                "name": "analyze_data",
                "description": "Use AI to analyze data from a table and answer questions",
                "parameters": {
                    "table_name": "string - table to analyze",
                    "question": "string - question about the data"
                }
            }
        ]
    
    def call_tool(self, tool_name: str, **kwargs):
        """Execute a tool by name"""
        if tool_name not in self.tools:
            return {"error": f"Tool {tool_name} not found"}
        return self.tools[tool_name](**kwargs)

# Create the server
server = SnowflakeToolServer("snowflake-server", SNOWFLAKE_CONFIG)
print(f"\n✅ MCP Server ready and waiting for clients")


PART 1: CREATING MCP SERVER

🏭 MCP SERVER 'snowflake-server' initializing...
   Registering 4 tools internally

✅ MCP Server ready and waiting for clients


In [32]:
print("\n" + "="*70)
print("PART 2: MCP CLIENT - DYNAMIC DISCOVERY")
print("="*70)

class MCPClient:
    """
    MCP Client - Discovers tools at RUNTIME
    
    Key Point: Client doesn't know tools beforehand!
    It asks servers: "What can you do?"
    """
    
    def __init__(self):
        self.servers = {}
        self.discovered_tools = []
        print("\n📬 MCP CLIENT: Initializing...")
        print("   Note: Client doesn't know ANY tools yet!")
    
    def connect(self, server_name: str, server):
        """Connect to an MCP server"""
        print(f"\n🔌 CLIENT: Connecting to '{server_name}'...")
        self.servers[server_name] = server
        print(f"   ✅ Connection established")
    
    def discover_all_tools(self):
        """
        🌟 DISCOVERY PHASE - This is the key!
        Client asks each server: "What capabilities do you have?"
        """
        print("\n" + "="*70)
        print("🔍 DISCOVERY PHASE - CLIENT ASKING SERVERS")
        print("="*70)
        
        all_tools = []
        
        for server_name, server in self.servers.items():
            print(f"\n📞 CLIENT → {server_name}: 'What tools do you offer?'")
            
            # This is the discovery call!
            tools = server.get_tool_descriptions()
            
            print(f"📥 CLIENT ← {server_name}: 'I have {len(tools)} tools:'")
            
            for tool in tools:
                tool['server'] = server_name  # Tag with server name
                all_tools.append(tool)
                print(f"      ✓ {tool['name']}: {tool['description']}")
        
        self.discovered_tools = all_tools
        
        print(f"\n{'='*70}")
        print(f"🎉 DISCOVERY COMPLETE!")
        print(f"   Total tools discovered: {len(all_tools)}")
        print(f"   From {len(self.servers)} server(s)")
        print("="*70)
        
        return all_tools
    
    def get_discovered_tools(self):
        """Return tools that were discovered"""
        return self.discovered_tools
    
    def call_tool(self, server_name: str, tool_name: str, arguments: dict):
        """Route tool call to the appropriate server"""
        print(f"\n📤 CLIENT: Routing request...")
        print(f"   → Server: {server_name}")
        print(f"   → Tool: {tool_name}")
        print(f"   → Arguments: {arguments}")
        
        if server_name not in self.servers:
            return {"error": "Server not found"}
        
        server = self.servers[server_name]
        result = server.call_tool(tool_name, **arguments)
        
        print(f"📥 CLIENT: Received response from server")
        return result

# Create client
client = MCPClient()

# Connect to server
client.connect("snowflake-server", server)

# 🌟 DISCOVERY HAPPENS HERE!
available_tools = client.discover_all_tools()


PART 2: MCP CLIENT - DYNAMIC DISCOVERY

📬 MCP CLIENT: Initializing...
   Note: Client doesn't know ANY tools yet!

🔌 CLIENT: Connecting to 'snowflake-server'...
   ✅ Connection established

🔍 DISCOVERY PHASE - CLIENT ASKING SERVERS

📞 CLIENT → snowflake-server: 'What tools do you offer?'

📋 SERVER: Responding to discovery request from client...
   Exposing 4 tool descriptions
📥 CLIENT ← snowflake-server: 'I have 4 tools:'
      ✓ query_database: Execute a SQL query on Snowflake database
      ✓ list_tables: List all tables in the database
      ✓ get_table_schema: Get the schema/structure of a specific table
      ✓ analyze_data: Use AI to analyze data from a table and answer questions

🎉 DISCOVERY COMPLETE!
   Total tools discovered: 4
   From 1 server(s)


In [33]:
print("\n" + "="*70)
print("PART 3: LLM BRAIN - PLANS WITH DISCOVERED TOOLS")
print("="*70)

class CortexBrain:
    """
    LLM Brain that makes decisions based on DISCOVERED tools
    Brain receives the list of tools that Client discovered
    """
    
    def __init__(self, sf_config: dict):
        self.config = sf_config
        print("\n🧠 BRAIN: Initialized (Snowflake Cortex)")
    
    def decide_tool(self, user_query: str, discovered_tools: list) -> dict:
        """
        LLM analyzes query and chooses from DISCOVERED tools
        """
        print(f"\n{'='*70}")
        print(f"🧠 BRAIN: Analyzing query")
        print("="*70)
        print(f"   Query: '{user_query}'")
        print(f"   Available tools (discovered): {len(discovered_tools)}")
        
        # Build detailed tool descriptions for LLM
        # LLM will reason ONLY from these descriptions (no hardcoded hints!)
        tools_text = "\n\n".join([
            f"Server: {t['server']}\n"
            f"Tool: {t['name']}\n"
            f"Description: {t['description']}\n"
            f"Parameters: {json.dumps(t['parameters'], indent=2)}"
            for t in discovered_tools
        ])
        
        prompt = f"""
You are an intelligent routing agent. A user has asked you a question.

Your job is to:
1. Analyze the user's query
2. Look at the available tools below
3. Choose the BEST tool that can answer the user's question
4. Determine the correct arguments for that tool

Available Tools (discovered at runtime):
{tools_text}

User Query: {user_query}

Think step by step:
- What is the user asking for?
- Which tool's description matches what they need?
- What parameters does that tool need?

Respond with ONLY a JSON object (no explanation, no markdown):
{{
    "server": "name-of-server",
    "tool_name": "name-of-tool",
    "arguments": {{"param_name": "param_value"}},
    "reasoning": "brief explanation of why this tool matches the user's need"
}}

IMPORTANT: Base your decision ONLY on the tool descriptions above. Do not assume tools exist that aren't listed.
"""
        
        try:
            conn = snowflake.connector.connect(**self.config)
            cursor = conn.cursor()
            
            cortex_query = f"""
            SELECT SNOWFLAKE.CORTEX.COMPLETE(
                'mistral-large2',
                '{prompt.replace("'", "''")}'
            ) as decision
            """
            
            cursor.execute(cortex_query)
            result = cursor.fetchone()
            decision_text = result[0] if result else "{}"
            
            cursor.close()
            conn.close()
            
            # Parse the decision
            decision_text = decision_text.strip()
            if "```json" in decision_text:
                decision_text = decision_text.split("```json")[1].split("```")[0]
            elif "```" in decision_text:
                decision_text = decision_text.split("```")[1].split("```")[0]
            
            decision = json.loads(decision_text.strip())
            
            print(f"\n🎯 BRAIN DECISION:")
            print(f"   Server: {decision.get('server')}")
            print(f"   Tool: {decision['tool_name']}")
            print(f"   Reasoning: {decision.get('reasoning', 'N/A')}")
            
            return decision
            
        except Exception as e:
            print(f"   ❌ Error in LLM: {str(e)}")
            return {
                "server": "snowflake-server",
                "tool_name": "list_tables",
                "arguments": {},
                "reasoning": f"Error occurred: {str(e)}"
            }

brain = CortexBrain(SNOWFLAKE_CONFIG)


PART 3: LLM BRAIN - PLANS WITH DISCOVERED TOOLS

🧠 BRAIN: Initialized (Snowflake Cortex)


In [34]:
print("\n" + "="*70)
print("PART 4: COMPLETE AI AGENT")
print("="*70)

class SnowflakeMCPAgent:
    """
    Complete AI Agent that uses:
    1. Brain (decides which tool)
    2. Client (routes to tool)
    3. Server (executes tool)
    
    Key: Agent works with DISCOVERED tools, not hardcoded!
    Agent ALWAYS gets fresh tools from client (not cached!)
    """
    
    def __init__(self, brain: CortexBrain, client: MCPClient):
        self.brain = brain
        self.client = client
        # ✅ Don't cache tools - always get fresh ones!
        
        print(f"\n🤖 AGENT: Initialized")
        print(f"   Brain: Snowflake Cortex")
        print(f"   Client: MCP Client")
        print(f"   Note: Agent will always use FRESH discovered tools")
    
    def process_query(self, user_query: str):
        """
        Complete flow:
        1. Get FRESH discovered tools from client
        2. Brain decides (based on current tools)
        3. Client routes
        4. Server executes
        """
        print("\n" + "="*70)
        print(f"🚀 AGENT: PROCESSING QUERY")
        print("="*70)
        print(f"Query: '{user_query}'")
        
        # ✅ ALWAYS get fresh tools from client
        current_tools = self.client.get_discovered_tools()
        print(f"Using {len(current_tools)} currently discovered tools")
        
        # Step 1: Brain decides using CURRENT tools
        decision = self.brain.decide_tool(user_query, current_tools)
        
        # Step 2: Client routes to the chosen tool
        result = self.client.call_tool(
            decision.get('server', 'snowflake-server'),
            decision['tool_name'],
            decision.get('arguments', {})
        )
        
        # Step 3: Display result
        print(f"\n{'='*70}")
        print(f"📊 FINAL RESULT:")
        print("="*70)
        
        if result.get('success'):
            if 'rows' in result:
                print(f"✅ Query returned {result['row_count']} rows")
                if result['row_count'] > 0:
                    print(f"Sample data:")
                    print(json.dumps(result['rows'][:2], indent=2, default=str))
            elif 'tables' in result:
                print(f"✅ Found {len(result['tables'])} tables:")
                for table in result['tables'][:5]:
                    print(f"   - {table}")
                if len(result['tables']) > 5:
                    print(f"   ... and {len(result['tables']) - 5} more")
            elif 'schema' in result:
                print(f"✅ Schema has {len(result['schema'])} columns:")
                for col in result['schema'][:5]:
                    print(f"   - {col['name']} ({col['type']})")
            elif 'analysis' in result:
                print(f"✅ Analysis:")
                print(f"{result['analysis'][:300]}...")
            else:
                print(json.dumps(result, indent=2))
        else:
            print(f"❌ Error: {result.get('error')}")
        
        print("="*70)
        return result

# Create agent
agent = SnowflakeMCPAgent(brain, client)


PART 4: COMPLETE AI AGENT

🤖 AGENT: Initialized
   Brain: Snowflake Cortex
   Client: MCP Client
   Note: Agent will always use FRESH discovered tools


In [35]:
agent.process_query("Show me all tables")


🚀 AGENT: PROCESSING QUERY
Query: 'Show me all tables'
Using 4 currently discovered tools

🧠 BRAIN: Analyzing query
   Query: 'Show me all tables'
   Available tools (discovered): 4

🎯 BRAIN DECISION:
   Server: snowflake-server
   Tool: list_tables
   Reasoning: The user is asking to see all tables, and the 'list_tables' tool matches this need.

📤 CLIENT: Routing request...
   → Server: snowflake-server
   → Tool: list_tables
   → Arguments: {}

🔧 TOOL EXECUTING: list_tables
   ✅ Found 7 tables
📥 CLIENT: Received response from server

📊 FINAL RESULT:
✅ Found 7 tables:
   - CHAT_MESSAGES
   - CHAT_SESSIONS
   - CHECKPOINTS
   - EPSS_DAILY
   - IPHONE_TABLE
   ... and 2 more


{'success': True,
 'tables': ['CHAT_MESSAGES',
  'CHAT_SESSIONS',
  'CHECKPOINTS',
  'EPSS_DAILY',
  'IPHONE_TABLE',
  'MEMORY_STORE',
  'SHORT_TERM_MEMORY']}

In [36]:
agent.process_query("What's the schema of CHAT_SESSIONS?")


🚀 AGENT: PROCESSING QUERY
Query: 'What's the schema of CHAT_SESSIONS?'
Using 4 currently discovered tools

🧠 BRAIN: Analyzing query
   Query: 'What's the schema of CHAT_SESSIONS?'
   Available tools (discovered): 4

🎯 BRAIN DECISION:
   Server: snowflake-server
   Tool: get_table_schema
   Reasoning: The user is asking for the schema of a specific table, which matches the description of the get_table_schema tool.

📤 CLIENT: Routing request...
   → Server: snowflake-server
   → Tool: get_table_schema
   → Arguments: {'table_name': 'CHAT_SESSIONS'}

🔧 TOOL EXECUTING: get_table_schema(CHAT_SESSIONS)
   ✅ Found 6 columns
📥 CLIENT: Received response from server

📊 FINAL RESULT:
✅ Schema has 6 columns:
   - SESSION_ID (VARCHAR(16777216))
   - USER_NAME (VARCHAR(16777216))
   - TITLE (VARCHAR(16777216))
   - CREATED_AT (TIMESTAMP_NTZ(9))
   - LAST_UPDATED (TIMESTAMP_NTZ(9))


{'success': True,
 'schema': [{'name': 'SESSION_ID',
   'type': 'VARCHAR(16777216)',
   'nullable': 'COLUMN'},
  {'name': 'USER_NAME', 'type': 'VARCHAR(16777216)', 'nullable': 'COLUMN'},
  {'name': 'TITLE', 'type': 'VARCHAR(16777216)', 'nullable': 'COLUMN'},
  {'name': 'CREATED_AT', 'type': 'TIMESTAMP_NTZ(9)', 'nullable': 'COLUMN'},
  {'name': 'LAST_UPDATED', 'type': 'TIMESTAMP_NTZ(9)', 'nullable': 'COLUMN'},
  {'name': 'MESSAGE_COUNT', 'type': 'NUMBER(38,0)', 'nullable': 'COLUMN'}]}

In [37]:
agent.process_query("SELECT * FROM IPHONE_TABLE LIMIT 5")


🚀 AGENT: PROCESSING QUERY
Query: 'SELECT * FROM IPHONE_TABLE LIMIT 5'
Using 4 currently discovered tools

🧠 BRAIN: Analyzing query
   Query: 'SELECT * FROM IPHONE_TABLE LIMIT 5'
   Available tools (discovered): 4

🎯 BRAIN DECISION:
   Server: snowflake-server
   Tool: query_database
   Reasoning: The user is asking to execute a specific SQL query, which matches the description of the query_database tool.

📤 CLIENT: Routing request...
   → Server: snowflake-server
   → Tool: query_database
   → Arguments: {'sql': 'SELECT * FROM IPHONE_TABLE LIMIT 5'}

🔧 TOOL EXECUTING: query_database
   SQL: SELECT * FROM IPHONE_TABLE LIMIT 5...
   ✅ Returned 5 rows
📥 CLIENT: Received response from server

📊 FINAL RESULT:
✅ Query returned 5 rows
Sample data:
[
  {
    "PRODUCTASIN": "B09G9BL5CP",
    "COUNTRY": "India",
    "DATE": "07-08-2024",
    "ISVERIFIED": false,
    "RATINGSCORE": 4,
    "REVIEWTITLE": "Camera is impressive but i am not happy with battery's",
    "REVIEWDESCRIPTION": null,
    

{'success': True,
 'rows': [{'PRODUCTASIN': 'B09G9BL5CP',
   'COUNTRY': 'India',
   'DATE': '07-08-2024',
   'ISVERIFIED': False,
   'RATINGSCORE': 4,
   'REVIEWTITLE': "Camera is impressive but i am not happy with battery's",
   'REVIEWDESCRIPTION': None,
   'REVIEWURL': 'https://www.amazon.in/gp/customer-reviews/R15DK15UHWE5WQ/ref=cm_cr_getr_d_rvw_ttl?ie=UTF8&ASIN=B09G93H3BR',
   'REVIEWEDIN': 'Reviewed in India on 7 August 2024',
   'VARIANT': 'Colour: BlueSize: 256 GB',
   'VARIANTASIN': 'B09G93H3BR',
   'REVIEW_EMBEDDINGS': None},
  {'PRODUCTASIN': 'B09G9BL5CP',
   'COUNTRY': 'India',
   'DATE': '29-10-2023',
   'ISVERIFIED': True,
   'RATINGSCORE': 4,
   'REVIEWTITLE': 'Bawal chij hai be..😀',
   'REVIEWDESCRIPTION': None,
   'REVIEWURL': 'https://www.amazon.in/gp/customer-reviews/R2LG7DDX5HRCIS/ref=cm_cr_getr_d_rvw_ttl?ie=UTF8&ASIN=B09V4MXBSN',
   'REVIEWEDIN': 'Reviewed in India on 29 October 2023',
   'VARIANT': 'Colour: GreenSize: 256 GB',
   'VARIANTASIN': 'B09V4MXBSN',
   'R

In [38]:
agent.process_query("Analyze the IPHONE_TABLE table and tell me the reviews")


🚀 AGENT: PROCESSING QUERY
Query: 'Analyze the IPHONE_TABLE table and tell me the reviews'
Using 4 currently discovered tools

🧠 BRAIN: Analyzing query
   Query: 'Analyze the IPHONE_TABLE table and tell me the reviews'
   Available tools (discovered): 4

🎯 BRAIN DECISION:
   Server: snowflake-server
   Tool: analyze_data
   Reasoning: The user is asking to analyze a specific table and get information about reviews. The 'analyze_data' tool matches this need as it can analyze data from a table and answer questions about it.

📤 CLIENT: Routing request...
   → Server: snowflake-server
   → Tool: analyze_data
   → Arguments: {'table_name': 'IPHONE_TABLE', 'question': 'tell me the reviews'}

🔧 TOOL EXECUTING: analyze_data
   Table: IPHONE_TABLE
   Question: tell me the reviews
   ✅ Analysis complete
📥 CLIENT: Received response from server

📊 FINAL RESULT:
✅ Analysis:
 Sure, here are the reviews from the provided sample data:

1. **Review 1:**
   - **PRODUCTASIN:** B09G9BL5CP
   - **COUNTRY:*

{'success': True,
 'analysis': ' Sure, here are the reviews from the provided sample data:\n\n1. **Review 1:**\n   - **PRODUCTASIN:** B09G9BL5CP\n   - **COUNTRY:** India\n   - **DATE:** 07-08-2024\n   - **ISVERIFIED:** false\n   - **RATINGSCORE:** 4\n   - **REVIEWTITLE:** "Camera is impressive but i am not happy with battery\'s"\n   - **REVIEWDESCRIPTION:** null\n   - **REVIEWURL:** [Amazon Review](https://www.amazon.in/gp/customer-reviews/R15DK15UHWE5WQ/ref=cm_cr_getr_d_rvw_ttl?ie=UTF8&ASIN=B09G93H3BR)\n   - **REVIEWEDIN:** Reviewed in India on 7 August 2024\n   - **VARIANT:** Colour: Blue, Size: 256 GB\n   - **VARIANTASIN:** B09G93H3BR\n   - **REVIEW_EMBEDDINGS:** null\n\n2. **Review 2:**\n   - **PRODUCTASIN:** B09G9BL5CP\n   - **COUNTRY:** India\n   - **DATE:** 29-10-2023\n   - **ISVERIFIED:** true\n   - **RATINGSCORE:** 4\n   - **REVIEWTITLE:** "Bawal chij hai be..😀"\n   - **REVIEWDESCRIPTION:** null\n   - **REVIEWURL:** [Amazon Review](https://www.amazon.in/gp/customer-reviews/R2L

In [39]:
print("=" * 70)
print("🧪 DEMO: What happens if we add a NEW server?")
print("=" * 70)

# Simulate adding a new server
class WeatherMCPServer:
    """A new server we're adding dynamically"""
    
    def __init__(self, name: str):
        self.name = name
        self.tools = {"get_weather": self.get_weather}
    
    def get_weather(self, city: str) -> dict:
        print(f"\n🔧 WEATHER TOOL: get_weather({city})")
        return {"success": True, "weather": "Sunny, 72°F", "city": city}
    
    def get_tool_descriptions(self) -> list:
        print(f"\n📋 WEATHER SERVER: Responding to discovery...")
        return [{
            "name": "get_weather",
            "description": "Get current weather for a city",
            "parameters": {"city": "string - city name"}
        }]
    
    def call_tool(self, tool_name: str, **kwargs):
        return self.tools[tool_name](**kwargs)

print("\n➕ Adding weather-server to the system...")
weather_server = WeatherMCPServer("weather-server")
client.connect("weather-server", weather_server)

print("\n🔍 Re-discovering tools (includes new server)...")
updated_tools = client.discover_all_tools()

🧪 DEMO: What happens if we add a NEW server?

➕ Adding weather-server to the system...

🔌 CLIENT: Connecting to 'weather-server'...
   ✅ Connection established

🔍 Re-discovering tools (includes new server)...

🔍 DISCOVERY PHASE - CLIENT ASKING SERVERS

📞 CLIENT → snowflake-server: 'What tools do you offer?'

📋 SERVER: Responding to discovery request from client...
   Exposing 4 tool descriptions
📥 CLIENT ← snowflake-server: 'I have 4 tools:'
      ✓ query_database: Execute a SQL query on Snowflake database
      ✓ list_tables: List all tables in the database
      ✓ get_table_schema: Get the schema/structure of a specific table
      ✓ analyze_data: Use AI to analyze data from a table and answer questions

📞 CLIENT → weather-server: 'What tools do you offer?'

📋 WEATHER SERVER: Responding to discovery...
📥 CLIENT ← weather-server: 'I have 1 tools:'
      ✓ get_weather: Get current weather for a city

🎉 DISCOVERY COMPLETE!
   Total tools discovered: 5
   From 2 server(s)


In [41]:
print("\n🧠 Brain now sees NEW weather tools (without code changes!)")
print(f"   Total tools now: {len(updated_tools)}")


🧠 Brain now sees NEW weather tools (without code changes!)
   Total tools now: 5


In [40]:
agent.process_query('What is the weather in Boston?')


🚀 AGENT: PROCESSING QUERY
Query: 'What is the weather in Boston?'
Using 5 currently discovered tools

🧠 BRAIN: Analyzing query
   Query: 'What is the weather in Boston?'
   Available tools (discovered): 5

🎯 BRAIN DECISION:
   Server: weather-server
   Tool: get_weather
   Reasoning: The user is asking for the current weather in Boston, which matches the description of the get_weather tool.

📤 CLIENT: Routing request...
   → Server: weather-server
   → Tool: get_weather
   → Arguments: {'city': 'Boston'}

🔧 WEATHER TOOL: get_weather(Boston)
📥 CLIENT: Received response from server

📊 FINAL RESULT:
{
  "success": true,
  "weather": "Sunny, 72\u00b0F",
  "city": "Boston"
}


{'success': True, 'weather': 'Sunny, 72°F', 'city': 'Boston'}